In [1]:
%pip install -U pip
%pip install -U aioboto3 pandas requests pillow boto3

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/47/6a/453160888fab7c6a432a6e25f8afe6256d0d9f2cbd25971021da6491d899/pip-23.3.1-py3-none-any.whl.metadata
  Using cached pip-23.3.1-py3-none-any.whl.metadata (3.5 kB)
Using cached pip-23.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.
  Using cached aioboto3-12.0.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached pandas-2.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached Pillow-10.1.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (9.5 kB)
  Using cached boto3-1.28.81-py3-none-any.whl.metadata (6.7 kB)
  Using cached aiobotocore-2.7.0-py3-none-any.whl.metadata (20 kB)
  Using cached botocore-1.31.64-py3-

In [1]:
from utils import queue_jobs, list_all_objects, get_file_from_bucket
import pandas as pd
import boto3
import itertools
import requests
from io import StringIO

asset_base_url = "https://salad-benchmark-assets.download/"
bucket_name = "salad-benchmark-public-assets"
clips_prefix = "cv-corpus-15.0-2023-09-08/en/clips/"
clip_duration_key = "cv-corpus-15.0-2023-09-08/en/clip_durations.tsv"

clip_duration_file = asset_base_url + clip_duration_key
file = requests.get(clip_duration_file).text
clip_durations = pd.read_csv(StringIO(file), sep="\t")

clip_durations.head()

,clip,duration[ms]
0,common_voice_en_139641.mp3,4992
1,common_voice_en_28076481.mp3,4644
2,common_voice_en_544777.mp3,4560
3,common_voice_en_31080299.mp3,5040
4,common_voice_en_149757.mp3,2064


In [3]:
def get_jobs():
    for i, row in clip_durations.iterrows():
        id = row["clip"]
        key = clips_prefix + id
        download_url = asset_base_url + key
        duration = row["duration[ms]"]
        yield {
            "id": id,
            "download_url": download_url,
            "duration": duration
        }

await queue_jobs("willow-whisper-large-v3", get_jobs(), concurrency=40, delay=0)

[{'Successful': [{'Id': 'e111324e-6495-4595-b7e6-d47f5dfdfa5f', 'MessageId': '4dbae4df-4010-4680-841e-a59cdc6efa5b', 'MD5OfMessageBody': '299576e9dfa9396e61e684ce1b5ca303', 'SequenceNumber': '18881811428586480128'}, {'Id': '58fba26b-7d3a-4fa2-9eea-33031bc569d3', 'MessageId': 'a9e2116d-7d69-45d8-8837-e4aea77b9393', 'MD5OfMessageBody': 'f86b12b9cfd1880fd9d5dd5e7c36a58f', 'SequenceNumber': '18881811428586480129'}, {'Id': '3796ae4d-cadc-42e3-a5a9-7f26de51f03c', 'MessageId': '3464e690-eccc-4be4-af86-66451f34d341', 'MD5OfMessageBody': 'bd14bb859e228bf34db5ed40021171af', 'SequenceNumber': '18881811428586480130'}, {'Id': '92c7f5eb-736f-4f41-bb19-b7e768e1fd1b', 'MessageId': 'acec11ce-778a-44e0-b7f6-0b08abf419e1', 'MD5OfMessageBody': '483e3309f2411f390219831444f5de85', 'SequenceNumber': '18881811428586480131'}, {'Id': 'ad800fe8-1208-4250-b962-f74d5f063252', 'MessageId': 'e14741a9-94f2-410a-a112-72d0cdbe8bb0', 'MD5OfMessageBody': '32cc262c069146118c05c652505b7562', 'SequenceNumber': '188818114285

CancelledError: 